Importing Libraries

In [1]:
import numpy as np
import pandas as pd
data=pd.read_csv('train.csv')

Importing Data (MNIST DATA)

In [2]:
x1=(data.iloc[:,1:785]).T
y1=(data.iloc[:,0])
y1=np.array(y1).reshape(1,42000)

One Hot Encoding

In [3]:
a=np.zeros((10,42000))
for i in range(0,42000):
    a[y1[0,i],i]=1
y1=a
m=y1.shape[1]
x1=np.array(x1)

Accuarcy Function

In [4]:
def accu(dlayer,y,m3):
    ab=np.array(dlayer.argmax(axis=0)).reshape(1,m3)
    m1=0
    cd=np.array(y.argmax(axis=0)).reshape(1,m3)
    for i in range(0,m3):
        if (ab[0,i]==cd[0,i]):
            m1=m1+1
    return (m1/m3)*100

Normalizing Data

In [5]:
range1=(np.array(np.max(x1,axis=1)-np.min(x1,axis=1)).reshape(784,1))+1
avg1=(1/42000)*(np.sum(x1,axis=1, keepdims=True))
x1=(x1-np.dot(avg1,np.ones((1,42000))))/np.dot(range1,np.ones((1,42000)))
x=x1[:,0:35000]
y=y1[:,0:35000]
m2=x.shape[1]

Defining Variables & Parameters

In [6]:
#defining layers 
alayer = x
blayer = np.zeros((150,m))
clayer = np.zeros((50,m))
dlayer = np.zeros((10,m))
thetaBA = (np.random.randn(150,784))*(np.sqrt(1/100))
bBA = np.zeros((150,1))
thetaCB = (np.random.randn(50,150))*(np.sqrt(1/50))
bCB=np.zeros((50,1))
thetaDC=(np.random.randn(10,50))*np.sqrt(1/10)
bDC=np.zeros((10,1))

beta1=0.9
beta2=0.999
epsilon=10**(-8)
keep_prob=0.9
alpha=0.1  # learning rate

#Parameters for Adam optimizer
vdw1 = np.zeros((thetaDC.shape))
sdw1 = np.ones((thetaDC.shape))
vdb1 = np.zeros((bDC.shape[0],1))
sdb1 = np.ones((bDC.shape[0],1))
vdw2 = np.zeros((thetaCB.shape))
sdw2 = np.ones((thetaCB.shape))
vdb2 = np.zeros((bCB.shape[0],1))
sdb2 = np.ones((bCB.shape[0],1))
vdw3 = np.zeros((thetaBA.shape))
sdw3 = np.ones((thetaBA.shape))
vdb3 = np.zeros((bBA.shape[0],1))
sdb3 = np.ones((bBA.shape[0],1))

# Training 

In [7]:
#forward propagation
for i in range(0,300):
    
    blayer=np.dot(thetaBA,alayer)+bBA
    blayer=np.where(blayer<0,0,blayer)
    d1=np.random.rand(blayer.shape[0],blayer.shape[1])  # Dropout Regularization
    d1= (d1<keep_prob)                                  # Dropout Regularization
    blayer=(blayer*d1)/(keep_prob)                      # Dropout Regularization
    
    clayer=np.dot(thetaCB,blayer)+bCB
    clayer=np.where(clayer<0,0,clayer)
    d2=np.random.rand(clayer.shape[0],clayer.shape[1])  # Dropout Regularization
    d2= (d2<keep_prob)                                  # Dropout Regularization
    clayer=(clayer*d2)/(keep_prob)                      # Dropout Regularization
    
    dlayer=1/(1+np.exp(-(np.dot(thetaDC,clayer)+bDC)))
    #t=np.exp(np.dot(thetaDC,clayer)+bDC)
    #dlayer=t/(np.sum(t,axis=0,keepdims=True))
    
    
# backward propagation
    deltaD = dlayer-y
    dwDC = np.dot(deltaD,clayer.T)*(1/m)
    dbDC = (1/m)*(np.sum(deltaD,axis=1,keepdims=True))

    
    vdw1 = (beta1*vdw1+(1-beta1)*dwDC)#/(1-(beta1**(i+1)))
    sdw1 = (beta2*sdw1+(1-beta2)*(dwDC**2))#/(1-(beta2**(i+1)))
    vdb1 = (beta1*vdb1+(1-beta1)*dbDC)#/(1-(beta1**(i+1)))
    sdb1 = (beta2*sdb1+(1-beta2)*(dbDC**2))#/(1-(beta2**(i+1)))
    
    thetaDC = thetaDC - alpha*(vdw1/((np.sqrt(sdw1))+epsilon))
    bDC = bDC - alpha*(vdb1/((np.sqrt(sdb1))+epsilon))
    
    A1 = np.dot(thetaDC.T,deltaD)
    A1 = (A1*d2)/(keep_prob)
    
    deltaC = A1*(np.where(clayer>0,1,clayer))
    
    dwCB = np.dot(deltaC,blayer.T)*(1/m)
    dbCB = (1/m)*(np.sum(deltaC,axis=1,keepdims=True))

    
    vdw2 = (beta1*vdw2+(1-beta1)*(dwCB))#/(1-(beta1**(i+1)))
    sdw2 = (beta2*sdw2+(1-beta2)*(dwCB**2))#/(1-(beta2**(i+1)))
    vdb2 = (beta1*vdb2+(1-beta1)*(dbCB))#/(1-(beta1**(i+1)))
    sdb2 = (beta2*sdb2+(1-beta2)*(dbCB**2))#/(1-(beta2**(i+1)))
    
    thetaCB = thetaCB-alpha*(vdw2/((np.sqrt(sdw2))+epsilon))
    bCB = bCB-alpha*(vdb2/((np.sqrt(sdb2))+epsilon))
    
    A2 = np.dot(thetaCB.T,deltaC)
    A2 = (A2*d1)/(keep_prob)
    
    deltaB = A2*(np.where(blayer>0,1,blayer))
    
    dwBA = np.dot(deltaB,alayer.T)*(1/m)
    dbBA = (1/m)*(np.sum(deltaB,axis=1,keepdims=True))

    
    vdw3 = (beta1*vdw3+(1-beta1)*(dwBA))#/(1-(beta1**(i+1)))
    sdw3 = (beta2*sdw3+(1-beta2)*(dwBA**2))#/(1-(beta2**(i+1)))
    vdb3 = (beta1*vdb3+(1-beta1)*(dbBA))#/(1-(beta1**(i+1)))
    sdb3 = (beta2*sdb3+(1-beta2)*(dbBA**2))#/(1-(beta2**(i+1)))
    
    thetaBA = thetaBA-alpha*(vdw3/((np.sqrt(sdw3))+epsilon))
    bBA = bBA-alpha*(vdb3/((np.sqrt(sdb3))+epsilon))
    ac=accu(dlayer,y,m2)
    print(' Accuracy : '+ str(ac),end=" ")
    

 Accuracy : 8.191428571428572  Accuracy : 8.27142857142857  Accuracy : 8.714285714285715  Accuracy : 9.305714285714286  Accuracy : 10.588571428571429  Accuracy : 11.914285714285715  Accuracy : 13.26857142857143  Accuracy : 14.737142857142857  Accuracy : 16.754285714285714  Accuracy : 18.10857142857143  Accuracy : 20.62857142857143  Accuracy : 22.825714285714284  Accuracy : 25.52857142857143  Accuracy : 28.305714285714284  Accuracy : 31.35142857142857  Accuracy : 33.900000000000006  Accuracy : 37.31428571428572  Accuracy : 39.76571428571429  Accuracy : 42.60857142857143  Accuracy : 44.902857142857144  Accuracy : 47.42285714285715  Accuracy : 49.22285714285714  Accuracy : 51.40285714285714  Accuracy : 53.142857142857146  Accuracy : 55.151428571428575  Accuracy : 56.34571428571429  Accuracy : 57.885714285714286  Accuracy : 59.01714285714286  Accuracy : 60.07714285714286  Accuracy : 61.291428571428575  Accuracy : 62.31142857142857  Accuracy : 63.09714285714285  Accuracy : 63.90857142857143

# Test Accuracy

In [8]:
m4=0
for i in range(0,42000):
    alayer=np.array(x1[:,i]).reshape(784,1)
    y_test=np.array(y1[:,i]).reshape(10,1)
    blayer=np.dot(thetaBA,alayer)+bBA
    blayer=np.where(blayer<0,0,blayer)
    
    clayer=np.dot(thetaCB,blayer)+bCB
    clayer=np.where(clayer<0,0,clayer)
    
    dlayer=1/(1+np.exp(-(np.dot(thetaDC,clayer)+bDC)))
          
    ab=np.array(dlayer.argmax())
    cd=np.array(y_test.argmax())
    if(ab==cd):
        m4=m4+1
print('Test Accuracy: ')
print((m4/42000)*100) 

Test Accuracy: 
92.16428571428571
